In [45]:
import numpy as np

In [46]:
def convert_vel_to_PWM(velocity):
    if velocity > 0:
        return min_pwm + velocity / cm_per_sec_per_PWM
    elif velocity < 0:
        return -min_pwm + velocity / cm_per_sec_per_PWM
    else:
        return 0

def convert_PWM_to_vel(PWM):
    if PWM > 0:
        return (PWM - min_pwm) * cm_per_sec_per_PWM
    elif PWM < 0:
        return (PWM + min_pwm) * cm_per_sec_per_PWM
    else:
        return 0

In [47]:

def x_ref_dumb_func(t):
        return np.array([12.5,0,0])
    
x_ref_func = x_ref_dumb_func
t = 1
dt = 1
x_act = np.array([0,0,0])
x_act_prev = np.array([0,0,0])
PWM_L_prev = 80
PWM_R_prev = 80

m = 0.83
cm_per_sec_per_PWM = 0.15
min_pwm = 70 #prev min_pwm

# mass of the robot (kilograms)
m = 0.830 #

# yoke point distance from center of the wheel base (centimeters)
r_length = 5 #prev r_length

K = -0.1  # spring constant
B = 0.8   # damper constant
I = 20  

In [48]:
    # Unit vector in direction of x_act
    x_unit_bot = np.array([np.cos(np.deg2rad(x_act[2])),
                           np.sin(np.deg2rad(x_act[2]))])

    # World  frame yoke vector
    r = x_unit_bot * r_length

    # World Frame location of the yoke point
    x_r = r + x_act[:2]

    # The target in the world frame
    x_ref = x_ref_func(t)[:2] + r

    # Distance vector from yoke to x_ref
    x_spring = x_ref[0:2] - x_r
    spring_displacement = np.linalg.norm(x_spring)

    # Unit vector in direction from yoke to x_ref
    x_unit_spring = x_spring / spring_displacement

    # TODO correct to using x_act_prev - x_act / timeslice
    #speed = (PWM_L_prev + PWM_R_prev) / 2
    delta_x_act = x_act_prev - x_act
    dot = np.dot(delta_x_act[:2], np.array(x_unit_spring))
    norm = np.linalg.norm(dot)
    speed = norm / dt

In [49]:
K = -0.00002

In [50]:
    # F_pd as given by the spring / damper function
    # F_pd = -K * x - B * x_dot
    F_pd = (-K * spring_displacement - B * speed) * x_unit_spring

    # F_trans = <F_pd, x_yoke_robot_frame>
    # F_trans / m = delta_PWM_trans
    r = x_unit_bot * r_length
    delta_vel_trans = np.dot(F_pd, r) / m

    # M_rot = r cross F_pd
    # delta_theta_dot = M_rot / I = r cross F_pd
    # delta_PWM_rot = delta_theta_dot * r_length
    M_rot = np.cross(r, F_pd)
    delta_vel_rot = r_length * M_rot / I

In [51]:
print(F_pd)
print(K)
print(delta_vel_rot)
print(delta_vel_trans)

[0.00025 0.     ]
-2e-05
0.0
0.0015060240963855422


In [52]:
    print(str(PWM_L_prev) + " " + str(PWM_R_prev))
    vel_L_prev = convert_PWM_to_vel(PWM_L_prev)
    vel_R_prev = convert_PWM_to_vel(PWM_R_prev)
    print(str(vel_L_prev) + " " + str(vel_R_prev))
    
    vel_L_new = vel_L_prev + delta_vel_trans - delta_vel_rot
    vel_R_new = vel_R_prev + delta_vel_trans + delta_vel_rot
    print(str(vel_L_new) + " " + str(vel_L_new))

80 80
1.5 1.5
1.5015060240963856 1.5015060240963856


In [18]:
    reduction_coefficient = 1
    max_vel = convert_PWM_to_vel(400)
    if abs(vel_L_new) > max_vel:
        reduction_coefficient = max_vel/abs(vel_L_new)
    if abs(vel_R_new) > max_vel:
        right_reduction_coefficient = max_vel/abs(vel_R_new)
        reduction_coefficient = min(right_reduction_coefficient, reduction_coefficient)

In [19]:

    PWM_L_new = convert_vel_to_PWM(vel_L_new)
    PWM_R_new = convert_vel_to_PWM(vel_R_new)

    PWM_L_new = int(reduction_coefficient * PWM_L_new)
    PWM_R_new = int(reduction_coefficient * PWM_R_new)

In [20]:
    def print_debug_info():
        print("{:>22} : {}".format("x_act", x_act))
        print("{:>22} : {}".format("x_ref", x_ref))
        print("{:>22} : {}".format("PWM_L_prev", PWM_L_prev))
        print("{:>22} : {}".format("PWM_R_prev", PWM_R_prev))
        print("{:>22} : {}".format("x_unit_bot", x_unit_bot))
        print("{:>22} : {}".format("x_r", x_r))
        print("{:>22} : {}".format("x_spring", x_spring))
        print("{:>22} : {}".format("spring_displacement", spring_displacement))
        print("{:>22} : {}".format("x_unit_spring", x_unit_spring))
        print("{:>22} : {}".format("speed", speed))
        print("{:>22} : {}".format("F_pd", F_pd))
        print("{:>22} : {}".format("delta_vel_trans", delta_vel_trans))
        print("{:>22} : {}".format("r", r))
        print("{:>22} : {}".format("M_rot", M_rot))
        print("{:>22} : {}".format("delta_vel_rot", delta_vel_rot))
        print("{:>22} : {}".format("vel_L_prev", vel_L_prev))
        print("{:>22} : {}".format("vel_R_prev", vel_R_prev))
        print("{:>22} : {}".format("vel_L_new", vel_L_new))
        print("{:>22} : {}".format("vel_R_new", vel_R_new))
        print("{:>22} : {}".format("PWM_L_new", PWM_L_new))
        print("{:>22} : {}".format("PWM_R_new", PWM_R_new))
        print("="*30)
    print_debug_info()

                 x_act : [0 0 0]
                 x_ref : [17.5  0. ]
            PWM_L_prev : 80
            PWM_R_prev : 80
            x_unit_bot : [1. 0.]
                   x_r : [5. 0.]
              x_spring : [12.5  0. ]
   spring_displacement : 12.5
         x_unit_spring : [1. 0.]
                 speed : 0.0
                  F_pd : [6.25 0.  ]
       delta_vel_trans : 37.65060240963856
                     r : [5. 0.]
                 M_rot : 0.0
         delta_vel_rot : 0.0
            vel_L_prev : 66.66666666666667
            vel_R_prev : 66.66666666666667
             vel_L_new : 104.31726907630522
             vel_R_new : 104.31726907630522
             PWM_L_new : 85
             PWM_R_new : 85
